In [16]:
#import libraries and modules
import mne
import numpy as np
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as  pd
import mne_bids
import string
import os
from os.path import join
from bids import BIDSLayout
sns.set(style="white", font_scale=1)

In [21]:
# Initiliaze the layout
data_path = os.path.join('/Users/alidzaye/rawdata')
layout = BIDSLayout(data_path)
print(layout)

BIDS Layout: .../Users/alidzaye/rawdata | Subjects: 8 | Sessions: 27 | Runs: 21


In [22]:
layout.get_subjects()

['001', '002', '003', '004', '005', '006', '007', '008']

In [23]:
layout.get_sessions()

['EphysMedOff01',
 'EphysMedOff02',
 'EphysMedOff03',
 'EphysMedOn01',
 'EphysMedOn02',
 'EphysMedOn03']

In [24]:
layout.get_tasks()

['BlockRotationL',
 'BlockRotationR',
 'BlockRotationWheel',
 'ContinuousStopping',
 'Evoked',
 'FreeDrawing',
 'ReadRelaxMoveL',
 'ReadRelaxMoveR',
 'Rest',
 'RestAndSpeech',
 'SelfpacedForceWheel',
 'SelfpacedRotationL',
 'SelfpacedRotationR',
 'SelfpacedSpeech',
 'Speech',
 'Transition',
 'UPDRSIII',
 'VigorStimL',
 'VigorStimR',
 'Visuomotor']

In [25]:
layout.get_acquisition()

['StimOff', 'StimOffOn', 'StimOn']

In [29]:
# Stim Rest data OFF & ON 
files_stimoff = layout.get(extension='vhdr', task='Rest', acquisition='StimOff', return_type='filename')
files_stimon = layout.get(extension='vhdr', task='Rest', acquisition='StimOn', return_type='filename')
files_stimoff_on = layout.get(extension='vhdr', task='Rest', acquisition='StimOffOn', return_type='filename')
print(len(files_stimoff))
print(len(files_stimon))
print(len(files_stimoff_on))

42
21
0


In [40]:
# Rest data OFF & ON StimOff
files = layout.get(extension='vhdr', task='Rest', acquisition='StimOff', return_type='filename')
files_off = layout.get(extension='vhdr', task='Rest', acquisition='StimOff', session=['EphysMedOff01','EphysMedOff02','EphysMedOff03'], return_type='filename')
files_on = layout.get(extension='vhdr', task='Rest', session=['EphysMedOn01','EphysMedOn02','EphysMedOn03'], acquisition='StimOff', return_type='filename')
print(len(files))
print(len(files_off))
print(len(files_on))

42
23
19


In [69]:
def read_files(files):
    '''
    reading Bids files
    '''
    raws=[]
    
    for index in range (len(files)):
        mne.io.read_raw(files[index])
        raws.append(mne.io.read_raw(files[index]))
        
    return raws

In [70]:
raws=read_files(files)

Extracting parameters from /Users/alidzaye/rawdata/sub-001/ses-EphysMedOff01/ieeg/sub-001_ses-EphysMedOff01_task-Rest_acq-StimOff_run-01_ieeg.vhdr...
Setting channel info structure...
Extracting parameters from /Users/alidzaye/rawdata/sub-001/ses-EphysMedOff01/ieeg/sub-001_ses-EphysMedOff01_task-Rest_acq-StimOff_run-01_ieeg.vhdr...
Setting channel info structure...
Extracting parameters from /Users/alidzaye/rawdata/sub-001/ses-EphysMedOff01/ieeg/sub-001_ses-EphysMedOff01_task-Rest_acq-StimOff_run-02_ieeg.vhdr...
Setting channel info structure...
Extracting parameters from /Users/alidzaye/rawdata/sub-001/ses-EphysMedOff01/ieeg/sub-001_ses-EphysMedOff01_task-Rest_acq-StimOff_run-02_ieeg.vhdr...
Setting channel info structure...
Extracting parameters from /Users/alidzaye/rawdata/sub-001/ses-EphysMedOn01/ieeg/sub-001_ses-EphysMedOn01_task-Rest_acq-StimOff_run-01_ieeg.vhdr...
Setting channel info structure...
Extracting parameters from /Users/alidzaye/rawdata/sub-001/ses-EphysMedOn01/ieeg/s

In [74]:
raws[3]

<RawBrainVision | sub-001_ses-EphysMedOn02_task-Rest_acq-StimOff_run-01_ieeg.eeg, 13 x 907572 (181.5 s), ~18 kB, data not loaded>

In [76]:
files_arr = np.array(files)

In [96]:
files[4]

'/Users/alidzaye/rawdata/sub-002/ses-EphysMedOff01/ieeg/sub-002_ses-EphysMedOff01_task-Rest_acq-StimOff_run-01_ieeg.vhdr'

In [129]:
#read the files with mne Bids
def read_BIDS_data(PATH_RUN, BIDS_PATH):
    """Given a run path and bids data path, read the respective data
    Parameters
    ----------
    PATH_RUN : string
    BIDS_PATH : string
    Returns
    -------
    raw_arr : mne.io.RawArray
    raw_arr_data : np.ndarray
    fs : int
    line_noise : int
    """
    PATH_BIDS = r"/Users/alidzaye/Charité - Universitätsmedizin Berlin/Interventional Cognitive Neuromodulation - Data/BIDS_Berlin_ECOG_LFP/rawdata"
    read_Bids_files = []
    for index in range (len(files)):
        PATH_RUN = files[index]
        entities = mne_bids.get_entities_from_fname(PATH_RUN)

        bids_path = mne_bids.BIDSPath(
            subject=entities["subject"],
            session=entities["session"],
            task=entities["task"],
            run=entities["run"],
            acquisition=entities["acquisition"],
            datatype="ieeg",
            root=BIDS_PATH,
                            )

        raw_arr = mne_bids.read_raw_bids(bids_path)
    read_Bids_files.append(raw_arr)
    return (
        read_Bids_files,
        raw_arr,
        raw_arr.get_data(),
        int(np.ceil(raw_arr.info["sfreq"])),
        int(raw_arr.info["line_freq"]),),

In [130]:
raw, dat, sfreq, line_freq = read_BIDS_data(PATH_RUN, PATH_BIDS)

Extracting parameters from /Users/alidzaye/Charité - Universitätsmedizin Berlin/Interventional Cognitive Neuromodulation - Data/BIDS_Berlin_ECOG_LFP/rawdata/sub-001/ses-EphysMedOff01/ieeg/sub-001_ses-EphysMedOff01_task-Rest_acq-StimOff_run-01_ieeg.vhdr...
Setting channel info structure...
Reading channel info from /Users/alidzaye/Charité - Universitätsmedizin Berlin/Interventional Cognitive Neuromodulation - Data/BIDS_Berlin_ECOG_LFP/rawdata/sub-001/ses-EphysMedOff01/ieeg/sub-001_ses-EphysMedOff01_task-Rest_acq-StimOff_run-01_channels.tsv.
Reading in coordinate system frame Other: None.
Reading electrode coords from /Users/alidzaye/Charité - Universitätsmedizin Berlin/Interventional Cognitive Neuromodulation - Data/BIDS_Berlin_ECOG_LFP/rawdata/sub-001/ses-EphysMedOff01/ieeg/sub-001_ses-EphysMedOff01_space-MNI152NLin2009bAsym_electrodes.tsv.
The read in electrodes file is: 
 [('name', ['ECOG_L_1_SMC_AT', 'ECOG_L_2_SMC_AT', 'ECOG_L_3_SMC_AT', 'ECOG_L_4_SMC_AT', 'ECOG_L_5_SMC_AT', '

<ipython-input-129-4219f154eb40>:31: RuntimeWarning: Did not find any events.tsv associated with sub-001_ses-EphysMedOff01_task-Rest_acq-StimOff_run-01.

The search_str was "/Users/alidzaye/Charité - Universitätsmedizin Berlin/Interventional Cognitive Neuromodulation - Data/BIDS_Berlin_ECOG_LFP/rawdata/sub-001/**/sub-001_ses-EphysMedOff01*events.tsv"
  raw_arr = mne_bids.read_raw_bids(bids_path)
<ipython-input-129-4219f154eb40>:31: RuntimeWarning: Defaulting coordinate frame to unknown from coordinate system input Other
  raw_arr = mne_bids.read_raw_bids(bids_path)
<ipython-input-129-4219f154eb40>:31: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_arr = mne_bids.read_raw_bids(bids_path)
<ipython-input-129-4219f154eb40>:31: RuntimeWarning: DigMontage is only a subset of info. There are 13 channel positions not present in the DigMontage. The required channels are:

['ECOG_L_1_2_SMC_AT', 'ECOG_L_1_3_SMC_AT', 'ECOG_L_1_4_SMC_AT', '

The read in electrodes file is: 
 [('name', ['ECOG_L_1_SMC_AT', 'ECOG_L_2_SMC_AT', 'ECOG_L_3_SMC_AT', 'ECOG_L_4_SMC_AT', 'ECOG_L_5_SMC_AT', 'ECOG_L_6_SMC_AT', 'LFP_R_1_STN_BS', 'LFP_R_2_STN_BS', 'LFP_R_3_STN_BS', 'LFP_R_4_STN_BS', 'LFP_R_5_STN_BS', 'LFP_R_6_STN_BS', 'LFP_R_7_STN_BS', 'LFP_R_8_STN_BS', 'LFP_L_1_STN_BS', 'LFP_L_2_STN_BS', 'LFP_L_3_STN_BS', 'LFP_L_4_STN_BS', 'LFP_L_5_STN_BS', 'LFP_L_6_STN_BS', 'LFP_L_7_STN_BS', 'LFP_L_8_STN_BS']), ('x', ['-39.5', '-41.4', '-41.5', '-41.5', '-40.5', '-40.0', '11.8327', '12.3707', '12.0276', '13.1289', '13.0713', '12.7332', '13.8241', '13.9119', '-12.3325', '-13.0522', '-13.5455', '-12.3516', '-13.7334', '-14.2192', '-13.0314', '-14.3399']), ('y', ['-74.0', '-65.0', '-54.5', '-44.5', '-34.0', '-24.0', '-14.8891', '-13.6485', '-14.668', '-14.5918', '-13.0653', '-14.108', '-14.0304', '-13.1935', '-17.1655', '-16.004', '-17.1799', '-16.6752', '-15.4365', '-16.6367', '-16.1272', '-15.5135']), ('z', ['40.5', '46.0', '52.0', '56.0', '59.5', '60.0

<ipython-input-129-4219f154eb40>:31: RuntimeWarning: Did not find any events.tsv associated with sub-002_ses-EphysMedOff02_task-Rest_acq-StimOff_run-01.

The search_str was "/Users/alidzaye/Charité - Universitätsmedizin Berlin/Interventional Cognitive Neuromodulation - Data/BIDS_Berlin_ECOG_LFP/rawdata/sub-002/**/sub-002_ses-EphysMedOff02*events.tsv"
  raw_arr = mne_bids.read_raw_bids(bids_path)
<ipython-input-129-4219f154eb40>:31: RuntimeWarning: iEEG Coordinate frame MNI152NLin2009bAsym is not a readable BIDS keyword by mne-bids yet. The allowed keywords are: ['ACPC', 'Pixels', 'Other']
  raw_arr = mne_bids.read_raw_bids(bids_path)
<ipython-input-129-4219f154eb40>:31: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_arr = mne_bids.read_raw_bids(bids_path)
<ipython-input-129-4219f154eb40>:31: RuntimeWarning: Did not find any events.tsv associated with sub-002_ses-EphysMedOff02_task-Rest_acq-StimOff_run-02.

The search_str was "/

Reading in coordinate system frame Other: None.
Reading electrode coords from /Users/alidzaye/Charité - Universitätsmedizin Berlin/Interventional Cognitive Neuromodulation - Data/BIDS_Berlin_ECOG_LFP/rawdata/sub-002/ses-EphysMedOff03/ieeg/sub-002_ses-EphysMedOff03_space-MNI152NLin2009bAsym_electrodes.tsv.
The read in electrodes file is: 
 [('name', ['ECOG_L_1_SMC_AT', 'ECOG_L_2_SMC_AT', 'ECOG_L_3_SMC_AT', 'ECOG_L_4_SMC_AT', 'ECOG_L_5_SMC_AT', 'ECOG_L_6_SMC_AT', 'LFP_R_1_STN_BS', 'LFP_R_2_STN_BS', 'LFP_R_3_STN_BS', 'LFP_R_4_STN_BS', 'LFP_R_5_STN_BS', 'LFP_R_6_STN_BS', 'LFP_R_7_STN_BS', 'LFP_R_8_STN_BS', 'LFP_L_1_STN_BS', 'LFP_L_2_STN_BS', 'LFP_L_3_STN_BS', 'LFP_L_4_STN_BS', 'LFP_L_5_STN_BS', 'LFP_L_6_STN_BS', 'LFP_L_7_STN_BS', 'LFP_L_8_STN_BS']), ('x', ['-39.5', '-41.4', '-41.5', '-41.5', '-40.5', '-40.0', '11.8327', '12.3707', '12.0276', '13.1289', '13.0713', '12.7332', '13.8241', '13.9119', '-12.3325', '-13.0522', '-13.5455', '-12.3516', '-13.7334', '-14.2192', '-13.0314', '-14.3399

...

Reading in coordinate system frame Other: None.
Reading electrode coords from /Users/alidzaye/Charité - Universitätsmedizin Berlin/Interventional Cognitive Neuromodulation - Data/BIDS_Berlin_ECOG_LFP/rawdata/sub-006/ses-EphysMedOn01/ieeg/sub-006_ses-EphysMedOn01_space-MNI152NLin2009bAsym_electrodes.tsv.
The read in electrodes file is: 
 [('name', ['LFP_R_1_STN_MT', 'LFP_R_2_STN_MT', 'LFP_R_3_STN_MT', 'LFP_R_4_STN_MT', 'LFP_R_5_STN_MT', 'LFP_R_6_STN_MT', 'LFP_R_7_STN_MT', 'LFP_R_8_STN_MT', 'LFP_L_1_STN_MT', 'LFP_L_2_STN_MT', 'LFP_L_3_STN_MT', 'LFP_L_4_STN_MT', 'LFP_L_5_STN_MT', 'LFP_L_6_STN_MT', 'LFP_L_7_STN_MT', 'LFP_L_8_STN_MT', 'ECOG_R_1_SMC_AT', 'ECOG_R_2_SMC_AT', 'ECOG_R_3_SMC_AT', 'ECOG_R_4_SMC_AT', 'ECOG_R_5_SMC_AT', 'ECOG_R_6_SMC_AT']), ('x', ['12.5437', '12.3982', '13.2188', '13.459', '12.893', '13.7156', '13.9535', '14.0308', '-11.3242', '-11.4257', '-12.5513', '-12.0225', '-12.1912', '-13.3134', '-12.7893', '-13.638', '27', '28.5', '29.5', '29.5', '30', '30']), ('y', ['-14.

<ipython-input-129-4219f154eb40>:31: RuntimeWarning: Defaulting coordinate frame to unknown from coordinate system input Other
  raw_arr = mne_bids.read_raw_bids(bids_path)
<ipython-input-129-4219f154eb40>:31: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_arr = mne_bids.read_raw_bids(bids_path)
<ipython-input-129-4219f154eb40>:31: RuntimeWarning: DigMontage is only a subset of info. There are 2 channel positions not present in the DigMontage. The required channels are:

['EEG_Cz_TM', 'EEG_Fz_TM'].

Consider using inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.
  raw_arr = mne_bids.read_raw_bids(bids_path)
<ipython-input-129-4219f154eb40>:31: RuntimeWarning: Did not find any events.tsv associated with sub-006_ses-EphysMedOn02_task-Rest_acq-StimOff_run-01.

The search_str was "/Users/alidzaye/Charité - Universitätsmedizin B

Reading channel info from /Users/alidzaye/Charité - Universitätsmedizin Berlin/Interventional Cognitive Neuromodulation - Data/BIDS_Berlin_ECOG_LFP/rawdata/sub-007/ses-EphysMedOn02/ieeg/sub-007_ses-EphysMedOn02_task-Rest_acq-StimOff_run-01_channels.tsv.
Reading in coordinate system frame Other: None.
Reading electrode coords from /Users/alidzaye/Charité - Universitätsmedizin Berlin/Interventional Cognitive Neuromodulation - Data/BIDS_Berlin_ECOG_LFP/rawdata/sub-007/ses-EphysMedOn02/ieeg/sub-007_ses-EphysMedOn02_space-MNI152NLin2009bAsym_electrodes.tsv.
The read in electrodes file is: 
 [('name', ['LFP_R_1_STN_MT', 'LFP_R_2_STN_MT', 'LFP_R_3_STN_MT', 'LFP_R_4_STN_MT', 'LFP_R_5_STN_MT', 'LFP_R_6_STN_MT', 'LFP_R_7_STN_MT', 'LFP_R_8_STN_MT', 'LFP_L_1_STN_MT', 'LFP_L_2_STN_MT', 'LFP_L_3_STN_MT', 'LFP_L_4_STN_MT', 'LFP_L_5_STN_MT', 'LFP_L_6_STN_MT', 'LFP_L_7_STN_MT', 'LFP_L_8_STN_MT', 'ECOG_R_1_SMC_AT', 'ECOG_R_2_SMC_AT', 'ECOG_R_3_SMC_AT', 'ECOG_R_4_SMC_AT', 'ECOG_R_5_SMC_AT', 'ECOG_R_6

<ipython-input-129-4219f154eb40>:31: RuntimeWarning: Did not find any events.tsv associated with sub-007_ses-EphysMedOn02_task-Rest_acq-StimOff_run-01.

The search_str was "/Users/alidzaye/Charité - Universitätsmedizin Berlin/Interventional Cognitive Neuromodulation - Data/BIDS_Berlin_ECOG_LFP/rawdata/sub-007/**/sub-007_ses-EphysMedOn02*events.tsv"
  raw_arr = mne_bids.read_raw_bids(bids_path)
<ipython-input-129-4219f154eb40>:31: RuntimeWarning: Defaulting coordinate frame to unknown from coordinate system input Other
  raw_arr = mne_bids.read_raw_bids(bids_path)
<ipython-input-129-4219f154eb40>:31: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_arr = mne_bids.read_raw_bids(bids_path)
<ipython-input-129-4219f154eb40>:31: RuntimeWarning: DigMontage is only a subset of info. There are 2 channel positions not present in the DigMontage. The required channels are:

['EEG_Cz_TM', 'EEG_Fz_TM'].

Consider using inst.set_channel_types i

ValueError: not enough values to unpack (expected 4, got 1)

In [128]:
read_Bids_files.

[]

In [103]:
raw.get_channel_types()

['eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'ecog',
 'ecog',
 'ecog',
 'ecog',
 'ecog',
 'ecog',
 'eeg',
 'eeg',
 'emg',
 'emg',
 'ecg',
 'misc',
 'misc',
 'misc',
 'misc',
 'misc',
 'misc']

In [7]:
raw.pick_types(ecog=True).get_channel_types()

['ecog', 'ecog', 'ecog', 'ecog', 'ecog', 'ecog']

In [8]:
raw.pick_types(ecog=True).ch_names

['ECOG_L_1_SMC_AT',
 'ECOG_L_2_SMC_AT',
 'ECOG_L_3_SMC_AT',
 'ECOG_L_4_SMC_AT',
 'ECOG_L_5_SMC_AT',
 'ECOG_L_6_SMC_AT']